# Implementing binary Decision Trees

**Goal:** to implement our own binary decision tree classifier.

Among other things, we have to:    
* Do Feature Engineering with SFrames, including transforming categorical variables into binary variables.
* Compute the number of misclassified examples ar an intermediate node.
* Use a function to find the best feature to split on.
* Build a binary decision tree from scratch.
* Make predictions using the decision tree and evaluate the accuracy of the decision tree.
* Visualize the decision at the root node.

**Important Note**: A Binary Decision Tree uses data containing only binary (0 or 1) features. This allows us to avoid dealing with:
* Multiple intermediate nodes in a split
* The thresholding issues of real-valued features.

In [1]:
import turicreate
import numpy as np

# Part 1. Preparing the Data

### Loading the lending club dataset

We use the [LendingClub](https://www.lendingclub.com/) dataset which contains real data on Loans. 

**About the data:**

The [LendingClub](https://www.lendingclub.com/) is a peer-to-peer lending company that connects borrowers and potential lenders/investors. We are looking to build a classification model to predict whether or not a loan provided by LendingClub is likely to [default](https://en.wikipedia.org/wiki/Default_%28finance%29).


In [2]:
loans = turicreate.SFrame('../data/lending-club-data.sframe/')

In [8]:
# Reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan, creating a new column `safe_loans`

loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)

loans = loans.remove_column('bad_loans')

We will just be using 4 categorical
features: 

1. Grade of the loan 
2. Length of the loan term (lending period)
3. Home ownership status: own, mortgage, or rent
4. Number of years of employment.

Since we are building a binary decision tree, we have to convert these categorical features to a binary representation. The method of choice will be 1-hot encoding.

In [9]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

# Subset original dataset to our working data
loans = loans[features + [target]]

Let's explore what the dataset looks like.

In [10]:
loans

grade,term,home_ownership,emp_length,safe_loans
B,36 months,RENT,10+ years,1
C,60 months,RENT,< 1 year,-1
C,36 months,RENT,10+ years,1
C,36 months,RENT,10+ years,1
A,36 months,RENT,3 years,1
E,36 months,RENT,9 years,1
F,60 months,OWN,4 years,-1
B,60 months,RENT,< 1 year,-1
C,60 months,OWN,5 years,1
B,36 months,OWN,10+ years,1


### Balancing the class distribution in the data

Most observations in the data are safe loans (around 81%). 
We will undersample the larger class (safe loans) in order to balance out our dataset, meaning we throw away many data points.
<br>
Use `seed=1` so you may get the same results.

In [11]:
safe_loans_all = loans[loans[target] == 1]
risky_loans_all = loans[loans[target] == -1]

ratio_risky_to_safe = len(risky_loans_all)/float(len(safe_loans_all))

safe_loans = safe_loans_all.sample(ratio_risky_to_safe, seed = 1)

risky_loans = risky_loans_all

# Merge both into a single sample
loans_data = risky_loans.append(safe_loans)

print("Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data)))
print("Percentage of risky loans                :", len(risky_loans) / float(len(loans_data)))
print("Total number of loans in our new dataset :", len(loans_data))

Percentage of safe loans                 : 0.5022361744216048
Percentage of risky loans                : 0.4977638255783951
Total number of loans in our new dataset : 46508


**Note:** More advances techniques for dealing with imbalanced data are covered in "[Learning from Imbalanced Data](http://www.ele.uri.edu/faculty/he/PDFfiles/ImbalancedLearning.pdf)" by Haibo He and Edwardo A. Garcia, *IEEE Transactions on Knowledge and Data Engineering* **21**(9) (June 26, 2009), p. 1263–1284.

### Transform categorical data into binary features

Since all of our features are currently categorical features, we want to turn them into binary features. 

For instance, the **home_ownership** (represents the home ownership status of the loanee), has three possible values: `own`, `mortgage` or `rent`. Hence, if a data point has as feature value: 
```
   {'home_ownership': 'RENT'}
```
we would turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

We are provided with the next block of code:

In [12]:
# For every feature of our working data:
for feature in features:
    
    # Convert the values in the column into single-key dictionaries 
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})  #SArray
    
    # Use the Sarray.unpack method to convert into an SFrame with a column for each value
    # an SArray of dictionaries will be expanded into as many columns as there are unique keys.
    # We use the original feature's name as a prefix
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature) # Sframe

    
    # Replace the default None values of the unpacked columns with 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)
    
    # Remove the original feature column and add the unpacked columns to the working data SFrame
    loans_data = loans_data.remove_column(feature)
    loans_data = loans_data.add_columns(loans_data_unpacked)

Let's see what the feature columns look like now:

In [13]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable from our list of features
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [14]:
print(f"Number of binary features = {len(features)}") # don't count the label column

Number of binary features = 25


Let's explore what one of these columns looks like:

In [15]:
# Verify feature columns are binary
loans_data['grade.A']

dtype: int
Rows: 46508
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, ... ]

This column is set to 1 if the loan grade is A and 0 otherwise.

### Train/Test split 

Split the data into a train-test split with 80% of data in the training set and 20% for the test set. Use `seed=1` so that everyone gets the same result.

In [16]:
train_data, test_data = loans_data.random_split(.8, seed=1)

# Part 2. Implementing a Decision Tree model

## Piece 1. Count number of mistakes from predicting Majority Class

A prediction at an intermediate node is done based on the **majority class** of the data points that belong to that node.

Our function should calculate the number of missclassified examples (**mistakes**) when predicting a **majority class**, so as to to help determine which feature is the best to split on at a given node of the tree.

Not that to compute the number of mistakes for a majority classifier, all we need is the label (y values) of the data points in the node. 

Now, let us write the function `node_mistakes` which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [17]:
'''
Computes the number of prediction mistakes at an intermediate node of a Decision Tree 
making predictions based on Majority Class.
'''
def nodeMistakes(labels):
    """Compute and return the number of misclassified examples of an intermediate node,
        given the array of labels of the data points in that node."""
    # Edge case: empty node
    if len(labels) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    n_safe = (labels == +1).sum()
    
    # Count the number of -1's (risky loans)
    n_risk = (labels == -1).sum()
                
    # The majority classifier labels all datapoints in the node with the most popular label.
    # Number of mistakes is equal to the observations not in the majority class.
    return min(n_safe, n_risk)

**Test our `nodeMistakes` function**

In [19]:
## TESTING THE `node_mistakes` FUNCTION

# Test case 1
example_labels = turicreate.SArray([-1, -1, 1, 1, 1])
if nodeMistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

# Test case 2
example_labels = turicreate.SArray([-1, -1, 1, 1, 1, 1, 1])
if nodeMistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 2 failed... try again!')

# Test case 3
example_labels = turicreate.SArray([-1, -1, -1, -1, -1, 1, 1])
if nodeMistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


## Piece 2. Function to pick best Feature to split on

**Classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

In [1]:
'''
Loops through the list of available features, 
computes classification error based on would-be feature split
and returns the feature with smallest classification error after split
'''
def bestFeatureSplit(data, features, target):
    """
    Receives:
    * data - SFrame that includes all the feature columns and label column
    * features - list of strings of column names to consider for splits
    * target - string for the name of target label column
    Returns: 
      string for the name of next best splitting feature
    
    """
    best_feature = None
    best_error = 10     # Error is always <= 1, we should intialize it with something larger than 1.

    N = float(len(data))
    
    # For every possible feature split
    for feature in features:
        
        # feature value = 0
        left_split = data[data[feature] == 0]
        
        # feature value = 1
        right_split =  data[data[feature] == 1]
            
        # Count number of misclassified examples
        left_mistakes = nodeMistakes(labels=left_split[target])           
        right_mistakes = nodeMistakes(labels=right_split[target])
            
        # Classification error of current split
        error = (left_mistakes + right_mistakes) / N 

        if error < best_error:
            best_feature = feature
            best_error = error
        
    
    return best_feature # Return the best feature found

**Test our `bestFeatureSplit` function**:

In [21]:
if bestFeatureSplit(train_data, features, 'safe_loans') == 'term. 36 months':
    print('Test passed!')
else:
    print('Test failed... try again!')

Test passed!


## Recursively Building the Decision Tree

Once we've correctly implemented pieces 1 and 2, we can get to the real thing. <br>
A generic Node in the decision tree will be represented as a dictionary described as follows:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : points to the left Node's dictionary
       'right'              : points to the right Node's dictionary
       'splitting_feature'  : The feature that this node splits on.
    }

In [22]:
'''
Creates a LEAF node given the array of target values of the data so as to set its prediction label.
'''
def create_leaf(labels):
    
    # Create a leaf node dictionary
    leaf = {
            'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':  True
            }
    
    # Count the number of datapoints from each class in this node
    num_plus = len(labels[labels == +1])
    num_minus = len(labels[labels == -1])
    
    # Set the prediction to be the majority class.
    if num_plus > num_minus:
        leaf['prediction'] = +1         
    else:
        leaf['prediction'] = -1        

        
    return leaf 

The next function learn a decision tree recursively, implements 3 stopping conditions:
1. All data points in a node are from the same class.
2. No more features to split on.
3. Based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process.

In [23]:
'''
Recursive function to build a decision tree that consists of dictionary nodes; 
given the data, list of features, target name, current tree depth and maximum depth limit.
'''
def buildTree(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a shallow copy of the features.
    
    labels = data[target]
    ###print("--------------------------------------------------------------------")
    ###print(f"Subtree: depth = {current_depth}, {len(labels)} data points.")

    # If all observations belong to the same class, we're done
    if  nodeMistakes(labels) == 0:
        ###print("Stopping condition 1 reached.")
        return create_leaf(labels)
    
    # If no remaining features to split on, we're done
    if len(remaining_features) == 0:   
        ###print("Stopping condition 2 reached.")
        return create_leaf(labels)    
    
    # If the max tree depth has been reached,
    if current_depth >= max_depth:
        ###print("Reached maximum depth. Stopping for now.")
        return create_leaf(labels)

    # Get the best feature to split on
    split_feature = bestFeatureSplit(data=data, features=features, target=target)
    
    # Make the splits 
    left_split = data[data[split_feature] == 0]
    right_split = data[data[split_feature] == 1]
    
    # Remove from the available features
    remaining_features.remove(split_feature)
    
    ###print(f"Split on feature {split_feature}. ({left_split}, {right_split})")
    
    # If the best feature does not split the data, return a leaf node with majority class prediction
    if len(left_split) == len(data):
        ###print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        ###print("Creating leaf node.")
        return create_leaf(right_split[target])


    # Else, recurse on left and right subtrees
    left_tree = buildTree(left_split, remaining_features, target, current_depth + 1, max_depth)        

    right_tree = buildTree(right_split, remaining_features, target, current_depth + 1, max_depth)
    
    # Return the current intermediate node
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': split_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Helper function to count the nodes in a tree:

In [24]:
'''Recursive function to count nodes in a tree'''
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return count_nodes(tree['left']) + count_nodes(tree['right']) + 1

**Test our `buildTree` function**

In [26]:
## TESTING OF `builTree` IMPLEMENTATION ##
features = loans_data.column_names()
features.remove('safe_loans')

small_data_decision_tree = buildTree(data=train_data, features=features, target='safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print('Test passed!')
else:
    print('Test failed... try again!')
    print('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print('Number of nodes that should be there : 13' )
    
## VERBOSE OUTPUT WARNING ##

Test passed!


## Train a Decision Tree Model

We now train a tree model on the **train_data**, limiting the depth to 6 (`max_depth = 6`) to make sure the algorithm doesn't run for too long. 

In [27]:
# Make sure to cap the depth at 6 by using max_depth = 6
my_decision_tree = buildTree(data=train_data, features=features, target='safe_loans', max_depth=6)

## Making predictions with a decision tree

We can make predictions from the decision tree with a simple recursive function that traverses from root to leaf. 

Below, we implement a function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that prints out to console the prediction path when set to `True`.

In [28]:
'''
Recursive function that traverses a Decision Tree to return the predicted label for a given input x.
'''
def classify(node, x, annotate = False):   
    
    if node['is_leaf']:
        if annotate: 
            print(f"Reached leaf, predicted: {node['prediction']}")
        return node['prediction'] 
    
    else:
        # Go down the subtree that corresponds to feature split
        split_feature_value = x[node['splitting_feature']]
        if annotate: 
            print(f"Split on {node['splitting_feature']} = {split_feature_value}")
        
        if split_feature_value == 0:  
            # belongs to left_split
            return classify(node['left'], x, annotate)
        else:
            # belongs to right_split
            return classify(node['right'], x, annotate)

Now, let's consider the first example of the test set and see what `my_decision_tree` model predicts for this data point.

In [29]:
test_data[0]

{'safe_loans': -1,
 'grade.A': 0,
 'grade.B': 0,
 'grade.C': 0,
 'grade.D': 1,
 'grade.E': 0,
 'grade.F': 0,
 'grade.G': 0,
 'term. 36 months': 0,
 'term. 60 months': 1,
 'home_ownership.MORTGAGE': 0,
 'home_ownership.OTHER': 0,
 'home_ownership.OWN': 0,
 'home_ownership.RENT': 1,
 'emp_length.1 year': 0,
 'emp_length.10+ years': 0,
 'emp_length.2 years': 1,
 'emp_length.3 years': 0,
 'emp_length.4 years': 0,
 'emp_length.5 years': 0,
 'emp_length.6 years': 0,
 'emp_length.7 years': 0,
 'emp_length.8 years': 0,
 'emp_length.9 years': 0,
 'emp_length.< 1 year': 0,
 'emp_length.n/a': 0}

In [30]:
print('Predicted class: %s ' % classify(my_decision_tree, test_data[0]))

Predicted class: -1 


Annotated path:

In [31]:
classify(my_decision_tree, test_data[0], annotate=True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
Reached leaf, predicted: -1


-1

**Quiz Question:** What was the feature that **my_decision_tree** first split on while making the prediction for test_data[0]?
`A = term. 36 months`

**Quiz Question:** What was the first feature that lead to a right split of test_data[0]? `A=grade.D`

**Quiz Question:** What was the last feature split on before reaching a leaf node for test_data[0]? `A=grade.D`

## Evaluating our Decision Tree model

Next we will compute the classification error of the tree on the given dataset.

 **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Now, write a function called `evaluate_classification_error` that takes in as input:
1. `tree` (as described above)
2. `data` (an SFrame)
3. `target` (a string - the name of the target/label column)

This function should calculate a prediction (class label) for each row in `data` using the decision `tree` and return the classification error computed using the above formula. Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [32]:
'''
Computes the Classification Error for a given decision tree, dataset, and target label.
'''
def evaluate_classification_error(tree, data, target):
    """
    Receives:
    * tree - the root node of a learned decision tree as implemented above
    * data - SFrame including feature columns and label column
    * target - name of target column
    """
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    
    # Once you've made the predictions, calculate the classification error and return it
    mistakes = (prediction != data[target]).sum()
    
    return mistakes / float(len(data))
    

Now, let's use this function to evaluate the classification error on the test set.

In [33]:
evaluate_classification_error(my_decision_tree, test_data, target)

0.3837785437311504

**Quiz Question:** Rounded to 2nd decimal point, what is the classification error of **my_decision_tree** on the **test_data**?
`A = 0.38`

# Extra: Printing out a decision stump

As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader). 

In [34]:
def print_stump(node, name = 'root'):
    split_name = node['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % node['prediction'])
        return None
    split_feature, split_value = split_name.split('.')
    print('                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(node['left']['prediction']) if node['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(node['right']['prediction']) if node['right']['is_leaf'] else 'subtree')))

In [35]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


**Quiz Question:** What is the feature that is used for the split at the root node? `A = term. 36 months`

#### Exploring the intermediate left subtree

Our tree implementation is a recursive dictionary, so we do have access to all the nodes. We can use
* `my_decision_tree['left']` to go left
* `my_decision_tree['right']` to go right

In [36]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the leftmost branch


In [37]:
# Left subtree
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.B == 0]               [grade.B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [39]:
# Left sub-subtree
print_stump(my_decision_tree['left']['left']['left'], my_decision_tree['left']['left']['splitting_feature'])

                       grade.B
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.C == 0]               [grade.C == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


**Quiz Question:** What is the path of the **first 3 feature splits** considered along the **left-most** branch of **my_decision_tree**?<br>
`A = term 36 months -> grade A -> grade B`

**Quiz Question:** What is the path of the **first 3 feature splits** considered along the **right-most** branch of **my_decision_tree**?<br>
`A = term 36 -> grade E -> grade F`

In [40]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.D == 0]               [grade.D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [45]:
print_stump(my_decision_tree['right']['right'], my_decision_tree['right']['splitting_feature'])

(leaf, label: -1)


In [48]:
print_stump(my_decision_tree['right']['left'], my_decision_tree['right']['left']['splitting_feature'])

                       grade.E
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.E == 0]               [grade.E == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [49]:
print_stump(my_decision_tree['right']['left']['left'], my_decision_tree['right']['left']['left']['splitting_feature'])

                       grade.F
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.F == 0]               [grade.F == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
